In [64]:
# importing necessary packages

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options

import time
import pprint
import pymongo
from pymongo import MongoClient
import pandas as pd

In [ ]:
# testing 1..2..3.....

In [65]:
SCROLL_PAUSE_TIME = 0.5

In [66]:
# creating a chrome drive instance
driver = webdriver.Chrome()

In [67]:
url = "https://en.wikipedia.org/wiki/List_of_London_Underground_stations"
driver.get(url)

In [68]:
table_element = driver.find_element(By.XPATH, '//*[@id="mw-content-text"]/div[1]/table[1]')
driver.execute_script("arguments[0].scrollIntoView({'block':'start', 'inline':'nearest'})", table_element)

In [69]:
soup = BeautifulSoup(driver.page_source, "html.parser")

In [70]:
names_table = soup.find_all('table', class_='wikitable sortable plainrowheaders jquery-tablesorter')

In [71]:
df = pd.read_html(str(names_table))
df = pd.DataFrame(df[0])

C:\Users\snkri\AppData\Local\Temp\ipykernel_20852\4268287041.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(names_table))


In [72]:
df.head()

,Station,Photograph,Line(s)[*],Local authority,Zone(s)[†],Opened[5],Main line opened,Other name(s)[note 2],Usage (millions per year)[6],Area Served
0,Acton Town,NaN,DistrictPiccadilly,Ealing,3,1 July 1879,NaN,Mill Hill Park: 1879–1910,2.90,Acton
1,Aldgate,NaN,Metropolitan[a]Circle,City of London,1,18 November 1876,NaN,NaN,3.53,Portsoken
2,Aldgate East,NaN,Hammersmith & City[d]District,Tower Hamlets,1,6 October 1884 (resited 31 October 1938),NaN,Commercial Road: (Proposed before opening),5.61,Whitechapel
3,Alperton,NaN,Piccadilly[h],Brent,4,28 June 1903,NaN,Perivale-Alperton: 1903–10,1.35,Alperton
4,Amersham,NaN,Metropolitan,Buckinghamshire,9,1 September 1892,NaN,Amersham: 1892–1922Amersham & Chesham Bois: 19...,0.95,Amersham


In [73]:
linkgen_url = 'https://media.star-telegram.com/static/labs/GoogleMapLink2/index.html'

In [74]:
df.iloc[63]

Station                                      Earl's Court
Photograph                                            NaN
Line(s)[*]                             DistrictPiccadilly
Local authority                    Kensington and Chelsea
Zone(s)[†]                                          1 & 2
Opened[5]                       (resited 1 February 1878)
Main line opened                                      NaN
Other name(s)[note 2]                                 NaN
Usage (millions per year)[6]                         8.40
Area Served                                  Earl's Court
Name: 63, dtype: object

In [75]:
df[df['Station'] == 'Ealing Common']

,Station,Photograph,Line(s)[*],Local authority,Zone(s)[†],Opened[5],Main line opened,Other name(s)[note 2],Usage (millions per year)[6],Area Served
62,Ealing Common,NaN,DistrictPiccadilly,Ealing,3,1 July 1879,NaN,Ealing Common: 1879–86Ealing Common and West A...,1.31,Ealing


In [76]:
driver.get(linkgen_url)

In [77]:

search_element = driver.find_element(By.XPATH, '//*[@id="autocomplete"]')

def link_generator(name):
        
        textToEnter = name + ' Underground station'
        
        driver.execute_script("arguments[0].value=\"" + textToEnter + "\"", search_element)

        submit = driver.find_element(By.XPATH, '/html/body/button')
        submit.click()

        soup = BeautifulSoup(driver.page_source, "html.parser")
        search_soup = soup.find(id="linkcopy")

        return search_soup.text

In [78]:
links = []

for name in df['Station']:
        links.append(link_generator(name))

In [80]:
pprint.pprint(links)

['https://www.google.com/maps?q=Acton+Town+Underground+station',
 'https://www.google.com/maps?q=Aldgate+Underground+station',
 'https://www.google.com/maps?q=Aldgate+East+Underground+station',
 'https://www.google.com/maps?q=Alperton+Underground+station',
 'https://www.google.com/maps?q=Amersham+Underground+station',
 'https://www.google.com/maps?q=Angel+Underground+station',
 'https://www.google.com/maps?q=Archway+Underground+station',
 'https://www.google.com/maps?q=Arnos+Grove+Underground+station',
 'https://www.google.com/maps?q=Arsenal+Underground+station',
 'https://www.google.com/maps?q=Baker+Street+Underground+station',
 'https://www.google.com/maps?q=Balham+Underground+station',
 'https://www.google.com/maps?q=Bank+Underground+station',
 'https://www.google.com/maps?q=Barbican+Underground+station',
 'https://www.google.com/maps?q=Barking+Underground+station',
 'https://www.google.com/maps?q=Barkingside+Underground+station',
 'https://www.google.com/maps?q=Barons+Court+Undergr

In [83]:
list_df = pd.DataFrame({'station_links' : links})

In [86]:
list_df.to_csv('C:/Users/snkri/OneDrive/Desktop/tfl project/tube_stations_list.csv')